In [1]:
# |Set parent to current working directory for imports
import os
import sys
from pathlib import Path

working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))

# |Import everything
from examples.CoDec_Research.code.pipelineOne.exp1_imports import *
from examples.CoDec_Research.code.pipelineOne.exp1_config import *

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-05-26 17:58:42.595607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748296722.613044  826034 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748296722.618022  826034 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748296722.632666  826034 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748296722.632695  826034 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748296722.632696  826034 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

Test code for bayesian search over parameter space

In [2]:
####################################################
################ SET EXP PARAMETERS ################
####################################################


# moving_veh_masks = get_mov_veh_masks(
#                                     training_config=training_config, 
#                                     device=device, 
#                                     dataset_path=dataset_path,
#                                     max_agents=moving_veh_count,
#                                     result_file_loc=simulation_results_path,
#                                     processID=processID
#                                     )


env_config, train_loader, env, sim_agent = get_gpuDrive_vars(
                                                            training_config=training_config,
                                                            device=device,
                                                            num_parallel_envs=num_parallel_envs,
                                                            dataset_path=dataset_path,
                                                            total_envs=total_envs,
                                                            sim_agent_path="daphne-cornelisse/policy_S10_000_02_27",
                                                            )



Initialized default environment


In [3]:
# |Check for saved construal utility values
for srFile in simulation_results_files:
    if "construal_vals" in srFile:
        with open(srFile, 'rb') as opn_file:
            default_values = pickle.load(opn_file)
        #2# |Ensure the correct file is being loaded
        if all(env_path2name(scene_path_) in default_values.keys() for scene_path_ in train_loader.dataset):
            print(f"Using construal values from file: {srFile}")
            break
        else:
            default_values = None
if default_values is None:
    raise FileNotFoundError("Could not find saved file for construal values for current scenes")

Using construal values from file: examples/CoDec_Research/results/simulation_results/Set3_construal_vals_2025-05-26 16:12:39.716309.pickle


In [ ]:
heuristic_params_str = '_'.join([heur_+str(param_) for heur_, param_ in heuristic_params.items()])

for srFile in simulation_results_files:
    if 'log_likelihood_DF' in srFile and heuristic_params_str in srFile and processID in srFile:
        construal_likelihoods = pd.read_csv(srFile, sep='\t')
        construal_likelihoods['base_construal'] = construal_likelihoods['base_construal'].map(eval) # String to tuple
        construal_likelihoods['test_construal'] = construal_likelihoods['test_construal'].map(eval) # String to tuple
        print(f"Using log-likelihood dataframe from file: {srFile}")
        break   

Using log-likelihood dataframe from file: examples/CoDec_Research/results/simulation_results/Set3_log_likelihood_DF_rel_heading0.0_cardinality2.tsv


In [ ]:

# |Set up functions
get_constral_heurisrtic_values_partial = partial(get_constral_heurisrtic_values, env=env, 
                                                 train_loader=train_loader, default_values=default_values)
target_param = "rel_heading"    # ego_distance or rel_heading
curr_heuristic_params = deepcopy(heuristic_params)


FileNotFoundError: [Errno 2] No such file or directory: 'examples/CoDec_Research/results/simulation_results/Set3_construal_action_likelihoods_rel_heading0.0_cardinality2.tsv'

In [ ]:
# |Define the black box function to optimize.
def black_box_function(lambda_heur):
    flag = False
    if isinstance(lambda_heur, list):
        lambda_heur = lambda_heur[0]
        flag = True
    # |lambda_heur: hyper parameter to optimize for.
    curr_heuristic_params[target_param] = lambda_heur
    curr_heuristic_values = get_constral_heurisrtic_values_partial(heuristic_params=curr_heuristic_params)
    # |Deep copy the dataframe them ke below changes
    curr_construal_likelihoods = deepcopy(construal_likelihoods)
    # |Convert log likelihoods to likelihoods
    curr_construal_likelihoods['traj_constr_likelihoods'] = np.exp(-1*curr_construal_likelihoods['-log_likelihood'])
    # |Get construal selection probs under lambda value
    counstral_probs = [curr_heuristic_values[row['scene']][eval(row['test_construal'])] for _, row in curr_construal_likelihoods.iterrows()]
    curr_construal_likelihoods['construal_probs'] = counstral_probs
    # |Get likelihood for trajectories given construals
    curr_construal_likelihoods['construal_likelihoods'] = curr_construal_likelihoods['construal_probs']*curr_construal_likelihoods['traj_constr_likelihoods']
    # |Group by trajectory and add 'construal_likelihoods' values
    traj_log_likelihoods = np.log(curr_construal_likelihoods.groupby(by=['scene','base_construal','sample']).sum()['construal_likelihoods'].to_list())
    # |Take product of all likelihoods (log sum)
    if flag:
        return -1*traj_log_likelihoods.sum().item()
    return traj_log_likelihoods.sum().item()





# |Create a BayesianOptimization optimizer, and optimize the given black_box_function.
pbounds = {"lambda_heur": [-15, 15]}    # Set range of lambda to optimize for.
optimizer = BayesianOptimization(f = black_box_function,
                                 pbounds = pbounds, verbose = 0,
                                 random_state = 4)
optimizer.maximize(init_points = 5, n_iter = 5)
print("Best result: {}; f(x) = {}.".format(optimizer.max["params"], optimizer.max["target"]))



# # |Create a BayesianOptimization optimizer,
# # |and optimize the given black_box_function.
# from skopt import gp_minimize
# from skopt.space import Real
# pbounds = [Real(-15,15,name="lambda_heur")]
# res_gp = gp_minimize(black_box_function,
#                         pbounds, n_calls = 15,
#                         random_state = 4)
# print("Best result: {}; f(x) = {}.".format(res_gp.x, res_gp.fun))

Processing Waymo batches: 100%|██████████| 1/1 [00:00<00:00,  5.82it/s]


Best result: {'lambda_heur': np.float64(-4.023554998190569)}; f(x) = -1075.8979389566657.


In [ ]:
curr_construal_likelihoods

,scene,base_construal,test_construal,sample,-log_likelihood,likelihoods,traj_likelihoods,construal_probs,construal_likelihoods
0,tfrecord-00136-of-00150_6,"(3,)","(1,)",0,162.181710,3.676025e-71,3.676025e-71,0.006327,2.325920e-73
1,tfrecord-00136-of-00150_6,"(3,)","(2,)",0,121.318475,2.051425e-53,2.051425e-53,0.232411,4.767743e-54
2,tfrecord-00136-of-00150_6,"(3,)","(3,)",0,49.007218,5.205178e-22,5.205178e-22,0.255345,1.329118e-22
3,tfrecord-00136-of-00150_6,"(3,)","(4,)",0,75.987852,9.974596e-34,9.974596e-34,0.287765,2.870341e-34
4,tfrecord-00136-of-00150_6,"(3,)","(5,)",0,50.851837,8.228639e-23,8.228639e-23,0.001939,1.595489e-25
...,...,...,...,...,...,...,...,...,...
763,tfrecord-00145-of-00150_75,"(4,)","(13,)",0,43.925818,8.380298e-20,8.380298e-20,0.003863,3.237118e-22
764,tfrecord-00145-of-00150_75,"(4,)","(14,)",0,43.925818,8.380298e-20,8.380298e-20,0.001319,1.105379e-22
765,tfrecord-00145-of-00150_75,"(4,)","(15,)",0,43.925818,8.380298e-20,8.380298e-20,0.162409,1.361034e-20
766,tfrecord-00145-of-00150_75,"(4,)","(16,)",0,43.925818,8.380298e-20,8.380298e-20,0.035137,2.944625e-21


# Sample Code

In [1]:
"""
    This part of pipeline 1 is dedicated to synthetic data generation. The code generates synthetic 
    data based on sampled construals (previous stage of pipeline).
"""

from copy import deepcopy
from functools import cache
from os import listdir
import json
import pickle
import gc
from datetime import datetime
from functools import partial

from scipy.special import softmax
import numpy as np
import math
from itertools import combinations

from typing import Any, List, Tuple
import time

import torch
import dataclasses


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))


# |GPUDrive imports
from gpudrive.utils.config import load_config
from examples.CoDec_Research.code.simulation.construal_main import generate_baseline_data, generate_selected_construal_traj, \
                                                                    get_constral_heurisrtic_values, generate_all_construal_trajnval
from examples.CoDec_Research.code.gpuDrive_utils import get_gpuDrive_vars, save_pickle
from examples.CoDec_Research.code.config import get_active_config


# Function to extract filename from path
env_path2name = lambda path: path.split("/")[-1].split(".")[0]


# |START TIMER
start_time = time.perf_counter()

####################################################
################ SET EXP PARAMETERS ################
####################################################

curr_config = get_active_config()

# Parameters for Inference
heuristic_params = {"ego_distance": 0.5, "cardinality": 1}              # Hueristics and their weight parameters (to be inferred)

construal_count_baseline = curr_config['construal_count_baseline']      # Number of construals to sample for baseline data generation
trajectory_count_baseline = curr_config['trajectory_count_baseline']    # Number of baseline trajectories to generate per construal


### Specify Environment Configuration ###

# |Location to store (and retrieve pre-computed) simulation results
simulation_results_path = "examples/CoDec_Research/results/simulation_results/"
simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]

# |Model Config (on which model was trained)
training_config = load_config("examples/experimental/config/reliable_agents_params")

# |Set scenario path
dataset_path = 'data/processed/construal/Set1V/'
processID = dataset_path.split('/')[-2]                 # Used for storing and retrieving relevant data

# |Set simulator config
max_agents = training_config.max_controlled_agents      # Get total vehicle count
num_parallel_envs = curr_config['num_parallel_envs']
total_envs = curr_config['total_envs']
device = eval(curr_config['device'])

# |Set construal config
construal_size = 1
observed_agents_count = max_agents - 1                              # Agents observed except self (used for vector sizes)
sample_size_utility = curr_config['sample_size_utility']            # Number of samples to compute expected utility of a construal

# |Other changes to variables
training_config.max_controlled_agents = 1                           # Control only the first vehicle in the environment
total_envs = min(total_envs, len(listdir(dataset_path)))


env_config, train_loader, env, sim_agent = get_gpuDrive_vars(
                                                            training_config=training_config,
                                                            device=device,
                                                            num_parallel_envs=num_parallel_envs,
                                                            dataset_path=dataset_path,
                                                            total_envs=total_envs,
                                                            sim_agent_path="daphne-cornelisse/policy_S10_000_02_27",
                                                            )



RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-05-14 17:11:01.013971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747257061.024993  116797 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747257061.028320  116797 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747257061.036986  116797 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747257061.037013  116797 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747257061.037014  116797 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

Initialized default environment


In [4]:


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default
import pickle

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))

In [5]:
srFile = "examples/CoDec_Research/results/simulation_results/Set3_baseline_state_action_pairs_2025-05-13 15:33:50.425015.pickle"
with open(srFile, 'rb') as opn_file:
    construal_action_likelihoods = pickle.load(opn_file)

In [33]:

srFile = "examples/CoDec_Research/results/simulation_results/Set3_baseline_state_action_pairs_2025-05-14 00:56:25.396045.pickle"
with open(srFile, 'rb') as opn_file:
    test_data = pickle.load(opn_file)